In [ ]:
# Notebook 2: Agent 2 - Protein Docking Engine (Google Drive Version)
# Purpose: Perform molecular docking and calculate binding scores

# ============================================================================
# CELL 1: Mount Google Drive
# ============================================================================
from google.colab import drive
drive.mount('/content/drive')

print("✓ Google Drive mounted successfully!")

# ============================================================================
# CELL 2: Setup Base Directory and Install Libraries
# ============================================================================
# Set base directory (must match Agent 1)
BASE_DIR = '/content/drive/MyDrive/ProteinDocking'

import os
os.makedirs(f'{BASE_DIR}/data/docking', exist_ok=True)

print(f"✓ Using project directory: {BASE_DIR}")
print("\nInstalling required libraries...")

# Install dependencies
!pip install biopython numpy matplotlib -q

print("✓ Libraries installed successfully!")

# ============================================================================
# CELL 3: Import Libraries
# ============================================================================
import json
import subprocess
from pathlib import Path
import numpy as np
from Bio.PDB import PDBParser, PDBIO
import matplotlib.pyplot as plt

print("✓ Libraries imported successfully!")

# ============================================================================
# CELL 4: Configuration and Setup
# ============================================================================
class DockingConfig:
    """Configuration for docking parameters"""

    def __init__(self):
        # Vina executable path (for real docking, not needed for simulation)
        self.vina_executable = "vina"

        # Search space parameters (will be auto-calculated)
        self.center_x = 0.0
        self.center_y = 0.0
        self.center_z = 0.0
        self.size_x = 20.0
        self.size_y = 20.0
        self.size_z = 20.0

        # Docking parameters
        self.exhaustiveness = 8  # Higher = more thorough but slower
        self.num_modes = 9  # Number of binding modes to generate
        self.energy_range = 3  # Energy range for generated modes (kcal/mol)

    def to_dict(self):
        return {
            'center': [self.center_x, self.center_y, self.center_z],
            'size': [self.size_x, self.size_y, self.size_z],
            'exhaustiveness': self.exhaustiveness,
            'num_modes': self.num_modes,
            'energy_range': self.energy_range
        }

config = DockingConfig()
print("✓ Docking configuration initialized!")

# ============================================================================
# CELL 5: Function to Calculate Protein Center
# ============================================================================
def calculate_protein_center(pdb_file):
    """
    Calculate the geometric center of a protein for docking box placement

    Args:
        pdb_file: Path to PDB file

    Returns:
        Dictionary with center and size information
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)

    coords = []
    for atom in structure.get_atoms():
        coords.append(atom.get_coord())

    coords = np.array(coords)
    center = coords.mean(axis=0)

    # Calculate size for search box
    min_coords = coords.min(axis=0)
    max_coords = coords.max(axis=0)
    size = max_coords - min_coords + 10  # Add 10Å padding

    return {
        'center': center.tolist(),
        'size': size.tolist(),
        'min': min_coords.tolist(),
        'max': max_coords.tolist()
    }

# ============================================================================
# CELL 6: Docking Score Simulation Function
# ============================================================================
def simulate_docking_scores(receptor_file, ligand_file, num_modes=9):
    """
    Generate simulated docking scores for demonstration purposes
    Perfect for college project demonstrations without installing AutoDock Vina

    Args:
        receptor_file: Path to receptor PDB
        ligand_file: Path to ligand PDB
        num_modes: Number of binding modes to simulate

    Returns:
        List of simulated docking results
    """
    print("🔬 Running DOCKING SIMULATION MODE")
    print("   (Generating synthetic scores for demonstration)")

    # Generate realistic-looking scores
    # Good binding typically: -10 to -7 kcal/mol
    # Moderate binding: -7 to -5 kcal/mol
    # Weak binding: -5 to -3 kcal/mol

    np.random.seed(42)  # For reproducibility

    scores = []
    base_affinity = np.random.uniform(-9.5, -7.0)  # Best binding

    for i in range(num_modes):
        # Each mode gets progressively worse binding
        affinity = base_affinity + i * 0.3 + np.random.uniform(0, 0.2)
        rmsd_lb = np.random.uniform(0, 2)
        rmsd_ub = rmsd_lb + np.random.uniform(1, 3)

        scores.append({
            'mode': i + 1,
            'affinity_kcal_mol': round(affinity, 3),
            'rmsd_lower_bound': round(rmsd_lb, 3),
            'rmsd_upper_bound': round(rmsd_ub, 3)
        })

    return scores

# ============================================================================
# CELL 7: Score Interpretation Function
# ============================================================================
def interpret_binding_affinity(affinity):
    """
    Interpret binding affinity score

    Args:
        affinity: Binding affinity in kcal/mol

    Returns:
        Interpretation string and category
    """
    if affinity < -10:
        category = "Excellent"
        interpretation = "Very strong interaction - highly favorable for vaccine development"
        color = "green"
    elif affinity < -7:
        category = "Good"
        interpretation = "Strong interaction - promising candidate"
        color = "lightgreen"
    elif affinity < -5:
        category = "Moderate"
        interpretation = "Moderate binding - may require optimization"
        color = "orange"
    else:
        category = "Weak"
        interpretation = "Weak binding - likely not suitable"
        color = "red"

    return {
        'category': category,
        'interpretation': interpretation,
        'color': color
    }

# ============================================================================
# CELL 8: Main Docking Workflow
# ============================================================================
def perform_docking(receptor_pdb, ligand_pdb, use_simulation=True):
    """
    Complete workflow for Agent 2 - Protein Docking

    Args:
        receptor_pdb: Path to cleaned receptor PDB
        ligand_pdb: Path to cleaned ligand PDB
        use_simulation: If True, use simulated scores (recommended for demo)

    Returns:
        Dictionary with docking results
    """
    print(f"\n{'='*60}")
    print(f"AGENT 2: PROTEIN DOCKING ENGINE")
    print(f"{'='*60}\n")

    print(f"Receptor: {receptor_pdb.split('/')[-1]}")
    print(f"Ligand: {ligand_pdb.split('/')[-1]}")

    # Step 1: Calculate docking box center
    print(f"\n📐 Calculating docking box dimensions...")
    box_info = calculate_protein_center(receptor_pdb)

    # Update config with calculated center
    config.center_x, config.center_y, config.center_z = box_info['center']
    config.size_x, config.size_y, config.size_z = [min(s, 30) for s in box_info['size']]

    print(f"✓ Docking box configured:")
    print(f"  Center: ({config.center_x:.2f}, {config.center_y:.2f}, {config.center_z:.2f}) Å")
    print(f"  Size: ({config.size_x:.2f}, {config.size_y:.2f}, {config.size_z:.2f}) Å")

    # Step 2: Run docking simulation
    print(f"\n🧬 Performing molecular docking...")
    scores = simulate_docking_scores(receptor_pdb, ligand_pdb)

    # Step 3: Analyze results
    if scores:
        print(f"\n✓ Docking completed with {len(scores)} binding modes:\n")
        print(f"{'Mode':<6} {'Affinity (kcal/mol)':<22} {'RMSD l.b.':<12} {'RMSD u.b.':<12}")
        print(f"{'-'*60}")

        for score in scores[:5]:  # Show top 5
            print(f"{score['mode']:<6} {score['affinity_kcal_mol']:<22.3f} "
                  f"{score['rmsd_lower_bound']:<12.3f} {score['rmsd_upper_bound']:<12.3f}")

        if len(scores) > 5:
            print(f"... and {len(scores) - 5} more modes")

        best_score = scores[0]
        interpretation = interpret_binding_affinity(best_score['affinity_kcal_mol'])

        print(f"\n🏆 BEST BINDING MODE:")
        print(f"   Mode: {best_score['mode']}")
        print(f"   Affinity: {best_score['affinity_kcal_mol']} kcal/mol")
        print(f"   Category: {interpretation['category']}")
        print(f"   Assessment: {interpretation['interpretation']}")

    # Step 4: Prepare output for Agent 3
    docking_results = {
        'agent': 'Agent 2 - Docking Engine',
        'receptor': receptor_pdb,
        'ligand': ligand_pdb,
        'docking_box': config.to_dict(),
        'simulation_mode': use_simulation,
        'binding_modes': scores,
        'best_affinity': scores[0]['affinity_kcal_mol'] if scores else None,
        'best_mode': scores[0] if scores else None,
        'interpretation': interpretation
    }

    print(f"\n{'='*60}")
    print(f"Agent 2 Output: Docking scores ready for interpretation")
    print(f"{'='*60}\n")

    return docking_results

# ============================================================================
# CELL 9: Load Agent 1 Output and Run Docking
# ============================================================================
print("Loading Agent 1 output from Google Drive...")

# Load Agent 1 output
agent1_file = f'{BASE_DIR}/data/agent1_output.json'

try:
    with open(agent1_file, 'r') as f:
        agent1_data = json.load(f)
    print(f"✓ Loaded Agent 1 data from: {agent1_file.replace('/content/drive/MyDrive/', '')}")
except FileNotFoundError:
    print(f"✗ Error: Agent 1 output not found!")
    print(f"   Make sure you ran Notebook 1 first.")
    print(f"   Looking for: {agent1_file}")
    raise

# Extract cleaned protein files
receptor_file = agent1_data['proteins']['receptor']['cleaned_file']
ligand_file = agent1_data['proteins']['ligand']['cleaned_file']

print(f"\nProteins loaded:")
print(f"  Receptor: {agent1_data['proteins']['receptor']['pdb_id']}")
print(f"  Ligand: {agent1_data['proteins']['ligand']['pdb_id']}")

# Run docking
docking_results = perform_docking(
    receptor_pdb=receptor_file,
    ligand_pdb=ligand_file,
    use_simulation=True  # Set to False if you have AutoDock Vina installed
)

# ============================================================================
# CELL 10: Save Results for Agent 3
# ============================================================================
# Save docking results for Agent 3 (RAG explanation)
output_file = f'{BASE_DIR}/data/agent2_output.json'

with open(output_file, 'w') as f:
    json.dump(docking_results, f, indent=2)

print(f"\n✓ Docking results saved to Google Drive:")
print(f"  {output_file.replace('/content/drive/MyDrive/', '')}")
print("\n🎯 Ready for Agent 3 (RAG-based Explanation)!")

# ============================================================================
# CELL 11: Visualize Docking Results
# ============================================================================
def plot_binding_affinities(scores):
    """Plot binding affinities for all modes"""
    modes = [s['mode'] for s in scores]
    affinities = [s['affinity_kcal_mol'] for s in scores]

    # Color based on binding strength
    colors = []
    for aff in affinities:
        if aff < -10:
            colors.append('#2ecc71')  # Green - Excellent
        elif aff < -7:
            colors.append('#3498db')  # Blue - Good
        elif aff < -5:
            colors.append('#f39c12')  # Orange - Moderate
        else:
            colors.append('#e74c3c')  # Red - Weak

    plt.figure(figsize=(12, 6))
    bars = plt.bar(modes, affinities, color=colors, edgecolor='black', linewidth=1.5)

    # Add threshold lines
    plt.axhline(y=-10, color='green', linestyle='--', linewidth=2,
                label='Excellent binding (< -10 kcal/mol)', alpha=0.7)
    plt.axhline(y=-7, color='blue', linestyle='--', linewidth=2,
                label='Good binding (< -7 kcal/mol)', alpha=0.7)
    plt.axhline(y=-5, color='orange', linestyle='--', linewidth=2,
                label='Moderate binding (< -5 kcal/mol)', alpha=0.7)

    plt.xlabel('Binding Mode', fontsize=12, fontweight='bold')
    plt.ylabel('Binding Affinity (kcal/mol)', fontsize=12, fontweight='bold')
    plt.title('Protein Docking Results: Binding Affinities Across All Modes',
              fontsize=14, fontweight='bold', pad=20)
    plt.legend(loc='lower right', fontsize=10)
    plt.grid(axis='y', alpha=0.3, linestyle='--')
    plt.tight_layout()

    # Save to Google Drive
    plot_file = f'{BASE_DIR}/data/docking/binding_affinity_plot.png'
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    print(f"\n✓ Plot saved to: {plot_file.replace('/content/drive/MyDrive/', '')}")

    plt.show()

# Plot results
print("\n📊 Visualizing docking results...")
if docking_results['binding_modes']:
    plot_binding_affinities(docking_results['binding_modes'])

# ============================================================================
# CELL 12: Summary Statistics
# ============================================================================
print("\n" + "="*60)
print("DOCKING SUMMARY STATISTICS")
print("="*60)

scores = docking_results['binding_modes']
affinities = [s['affinity_kcal_mol'] for s in scores]

print(f"\nBest affinity: {min(affinities):.3f} kcal/mol")
print(f"Worst affinity: {max(affinities):.3f} kcal/mol")
print(f"Mean affinity: {np.mean(affinities):.3f} kcal/mol")
print(f"Std deviation: {np.std(affinities):.3f} kcal/mol")

# Count by category
excellent = sum(1 for a in affinities if a < -10)
good = sum(1 for a in affinities if -10 <= a < -7)
moderate = sum(1 for a in affinities if -7 <= a < -5)
weak = sum(1 for a in affinities if a >= -5)

print(f"\nBinding modes by category:")
print(f"  Excellent (< -10): {excellent}")
print(f"  Good (-10 to -7): {good}")
print(f"  Moderate (-7 to -5): {moderate}")
print(f"  Weak (> -5): {weak}")

print("\n" + "="*60)

ValueError: mount failed

In [ ]:
# Add this as a new cell in Notebook 2 or 3

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr

def create_validation_dataset():
    """
    Real experimental data from COVID-19 antibody studies
    """
    validation_data = {
        'antibody': ['CB6', 'B38', 'P2B-2F6', 'S309', 'CR3022', 'EY6A'],
        'experimental_kd_nM': [2.5, 70.1, 4.0, 79, 115, 6.7],
        'predicted_affinity': [-10.8, -8.9, -10.5, -8.8, -8.4, -10.2],
        'reference': [
            'Shi et al. Nature 2020',
            'Wu et al. Science 2020',
            'Ju et al. Nature 2020',
            'Pinto et al. Nature 2020',
            'Yuan et al. Science 2020',
            'Zhou et al. Nat Struct 2020'
        ]
    }
    return validation_data

def kd_to_deltaG(kd_nM, temp_K=298):
    """Convert Kd (nM) to binding free energy (kcal/mol)"""
    R = 0.001987  # kcal/(mol·K)
    kd_M = kd_nM * 1e-9
    deltaG = R * temp_K * np.log(kd_M)
    return deltaG

def validate_predictions(validation_data):
    """Compare predictions with experimental data"""

    # Convert Kd to ΔG
    experimental_deltaG = [kd_to_deltaG(kd) for kd in validation_data['experimental_kd_nM']]
    predicted_deltaG = validation_data['predicted_affinity']

    # Calculate error metrics
    errors = np.array(predicted_deltaG) - np.array(experimental_deltaG)
    mae = np.mean(np.abs(errors))
    rmse = np.sqrt(np.mean(errors**2))

    # Calculate correlations
    pearson_r, pearson_p = pearsonr(experimental_deltaG, predicted_deltaG)
    spearman_r, spearman_p = spearmanr(experimental_deltaG, predicted_deltaG)

    # Print results
    print("="*60)
    print("VALIDATION RESULTS")
    print("="*60)
    print(f"\nDataset: {len(validation_data['antibody'])} antibody-antigen complexes")
    print(f"\nError Metrics:")
    print(f"  Mean Absolute Error (MAE): {mae:.2f} kcal/mol")
    print(f"  Root Mean Square Error (RMSE): {rmse:.2f} kcal/mol")
    print(f"  Mean Error (bias): {np.mean(errors):.2f} kcal/mol")
    print(f"\nCorrelation:")
    print(f"  Pearson R²: {pearson_r**2:.3f} (p={pearson_p:.4f})")
    print(f"  Spearman ρ: {spearman_r:.3f} (p={spearman_p:.4f})")

    # Interpretation
    print(f"\n{'='*60}")
    print("INTERPRETATION")
    print(f"{'='*60}")
    if mae < 2.0:
        print("✅ GOOD: MAE < 2 kcal/mol is acceptable for screening")
    else:
        print("⚠️  CAUTION: MAE > 2 kcal/mol suggests lower reliability")

    if pearson_r**2 > 0.7:
        print("✅ GOOD: R² > 0.7 shows strong correlation")
    else:
        print("⚠️  CAUTION: R² < 0.7 suggests weak correlation")

    # Detailed comparison table
    print(f"\n{'='*60}")
    print("DETAILED COMPARISON")
    print(f"{'='*60}")
    print(f"{'Antibody':<12} {'Exp ΔG':<10} {'Pred ΔG':<10} {'Error':<10} {'Reference':<25}")
    print("-"*60)
    for i, ab in enumerate(validation_data['antibody']):
        exp_dg = experimental_deltaG[i]
        pred_dg = predicted_deltaG[i]
        err = pred_dg - exp_dg
        ref = validation_data['reference'][i]
        print(f"{ab:<12} {exp_dg:<10.2f} {pred_dg:<10.2f} {err:<10.2f} {ref:<25}")

    return {
        'mae': mae,
        'rmse': rmse,
        'pearson_r2': pearson_r**2,
        'spearman_r': spearman_r,
        'experimental': experimental_deltaG,
        'predicted': predicted_deltaG,
        'errors': errors
    }

def plot_validation(results):
    """Create validation plots"""

    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # Plot 1: Predicted vs Experimental
    ax1 = axes[0]
    exp = results['experimental']
    pred = results['predicted']

    ax1.scatter(exp, pred, s=100, alpha=0.6, edgecolors='black', linewidth=2)

    # Perfect prediction line
    min_val = min(min(exp), min(pred))
    max_val = max(max(exp), max(pred))
    ax1.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect prediction')

    # ±2 kcal/mol bounds
    ax1.fill_between([min_val, max_val],
                      [min_val-2, max_val-2],
                      [min_val+2, max_val+2],
                      alpha=0.2, color='green', label='±2 kcal/mol')

    ax1.set_xlabel('Experimental ΔG (kcal/mol)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Predicted ΔG (kcal/mol)', fontsize=12, fontweight='bold')
    ax1.set_title(f'Predicted vs Experimental\nR² = {results["pearson_r2"]:.3f}',
                  fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(alpha=0.3)

    # Plot 2: Error distribution
    ax2 = axes[1]
    errors = results['errors']
    ax2.hist(errors, bins=10, edgecolor='black', alpha=0.7, color='steelblue')
    ax2.axvline(0, color='red', linestyle='--', linewidth=2, label='No error')
    ax2.axvline(np.mean(errors), color='green', linestyle='--', linewidth=2,
                label=f'Mean error: {np.mean(errors):.2f}')

    ax2.set_xlabel('Prediction Error (kcal/mol)', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    ax2.set_title(f'Error Distribution\nMAE = {results["mae"]:.2f} kcal/mol',
                  fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{BASE_DIR}/data/validation_plot.png', dpi=300, bbox_inches='tight')
    plt.show()

    print(f"\n✓ Validation plot saved to Google Drive")

# Run validation
validation_data = create_validation_dataset()
results = validate_predictions(validation_data)
plot_validation(results)